In [1]:
import numpy as np # linear algebra
import pandas as pd # CSV file I/O (e.g. pd.read_csv)
import os # reading the input files we have access to

print(os.listdir('.'))
TRAIN_PATH = 'train.csv'

['.ipynb', '.ipynb_checkpoints', 'GCP-Coupons-Instructions.rtf', 'Subm_1.csv', 'sample_submission.csv', 'submission.csv', 'taxi.ipynb', 'test.csv', 'train.csv']


In [2]:
"""%%time

s = !wc -l {TRAIN_PATH}

# add one because the file isn't quite correctly formatted as a CSV, should have a final newline char
n_rows = int(s[0].split(' ')[0])+1

print ('Exact number of rows: {%d}' % n_rows)"""

"%%time\n\ns = !wc -l {TRAIN_PATH}\n\n# add one because the file isn't quite correctly formatted as a CSV, should have a final newline char\nn_rows = int(s[0].split(' ')[0])+1\n\nprint ('Exact number of rows: {%d}' % n_rows)"

## Read training data

In [26]:
train_df =  pd.read_csv('train.csv', nrows =200000).dropna()
train_df.dtypes

key                   object
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

In [27]:
train_df.head(3)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2


In [28]:
test_df = pd.read_csv('test.csv')
test_df.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1


## Drop missing value

In [29]:
#np.where(train_df.isna())

In [30]:
train_df.describe(percentiles=[])

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,199999.000000,199999.000000,199999.000000,199999.000000,199999.000000,199999.000000
mean,11.342871,-72.506114,39.922322,-72.518673,39.925579,1.682453
std,9.837879,11.608125,10.048972,10.724226,6.751120,1.306727
min,-44.900000,-736.550000,-3116.285383,-1251.195890,-1189.615440,0.000000
50%,8.500000,-73.981743,40.752762,-73.980072,40.753225,1.000000
max,500.000000,2140.601160,1703.092772,40.851027,404.616667,6.000000


## Feature manipulation

In [31]:
train_df['pickup_datetime'] = train_df['pickup_datetime'].str.slice(0, 19)
train_df['pickup_datetime'] = pd.to_datetime(train_df['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')

In [32]:
train_df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.50,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.90,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.70,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.70,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.30,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1
5,2011-01-06 09:50:45.0000002,12.10,2011-01-06 09:50:45,-74.000964,40.731630,-73.972892,40.758233,1
6,2012-11-20 20:35:00.0000001,7.50,2012-11-20 20:35:00,-73.980002,40.751662,-73.973802,40.764842,1
7,2012-01-04 17:22:00.00000081,16.50,2012-01-04 17:22:00,-73.951300,40.774138,-73.990095,40.751048,1
8,2012-12-03 13:10:00.000000125,9.00,2012-12-03 13:10:00,-74.006462,40.726713,-73.993078,40.731628,1
9,2009-09-02 01:11:00.00000083,8.90,2009-09-02 01:11:00,-73.980658,40.733873,-73.991540,40.758138,2


In [33]:
train_df.columns

Index(['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count'],
      dtype='object')

In [34]:
mask = train_df['pickup_longitude'].between(-100, -40)
mask &= train_df['dropoff_longitude'].between(-100, -40)
mask &= train_df['pickup_latitude'].between(20, 60)
mask &= train_df['dropoff_latitude'].between(20, 60)
mask &= train_df['passenger_count'].between(0, 8)
mask &= train_df['fare_amount'].between(0, 250)
train_df = train_df[mask]

In [35]:
coord = ['pickup_longitude','pickup_latitude', 
          'dropoff_longitude', 'dropoff_latitude']

## Feature generation

In [36]:
#radius of earth in kilometers
R = 6373.0

pickup_lat  = np.radians(train_df["pickup_latitude"])
pickup_lon  = np.radians(train_df["pickup_longitude"])
dropoff_lat = np.radians(train_df["dropoff_latitude"])
dropoff_lon = np.radians(train_df["dropoff_longitude"])

dist_lon = dropoff_lon - pickup_lon
dist_lat = dropoff_lat - pickup_lat

#Formula
a = (np.sin(dist_lat/2))**2 + np.cos(pickup_lat) * np.cos(dropoff_lat) * (np.sin(dist_lon/2))**2 
c = 2 * np.arctan2( np.sqrt(a), np.sqrt(1-a)) 
d = R * c #(where R is the radius of the Earth)

train_df["trip_distance_km"] = d
train_df["trip_distance_2norm"] = np.sqrt(dist_lon**2+dist_lat**2)
train_df["trip_distance_1norm"] = abs(dist_lon)+abs(dist_lat)

#create new variable log of distance
train_df["log_trip_ditance"] = np.log(train_df["trip_distance_km"])

train_df[coord + ["trip_distance_km"]].head(7)

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_distance_km
0,-73.844311,40.721319,-73.841610,40.712278,1.031088
1,-74.016048,40.711303,-73.979268,40.782004,8.452786
2,-73.982738,40.761270,-73.991242,40.750562,1.389961
3,-73.987130,40.733143,-73.991567,40.758092,2.800149
4,-73.968095,40.768008,-73.956655,40.783762,1.999784
5,-74.000964,40.731630,-73.972892,40.758233,3.788428
6,-73.980002,40.751662,-73.973802,40.764842,1.556295


In [37]:
train_df.head(3)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance_km,trip_distance_2norm,trip_distance_1norm,log_trip_ditance
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1,1.031088,0.000165,0.000205,0.030614
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1,8.452786,0.001391,0.001876,2.134496
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,1.389961,0.000239,0.000335,0.329276


In [38]:
#radius of earth in kilometers
R = 6373.0

pickup_lat  = np.radians(test_df["pickup_latitude"])
pickup_lon  = np.radians(test_df["pickup_longitude"])
dropoff_lat = np.radians(test_df["dropoff_latitude"])
dropoff_lon = np.radians(test_df["dropoff_longitude"])

dist_lon = dropoff_lon - pickup_lon
dist_lat = dropoff_lat - pickup_lat

#Formula
a = (np.sin(dist_lat/2))**2 + np.cos(pickup_lat) * np.cos(dropoff_lat) * (np.sin(dist_lon/2))**2 
c = 2 * np.arctan2( np.sqrt(a), np.sqrt(1-a)) 
d = R * c #(where R is the radius of the Earth)

test_df["trip_distance_km"] = d
test_df["trip_distance_2norm"] = np.sqrt(dist_lon**2+dist_lat**2)
test_df["trip_distance_1norm"] = abs(dist_lon)+abs(dist_lat)

#create new variable log of distance
test_df["log_trip_ditance"] = np.log(test_df["trip_distance_km"])

test_df[coord + ["trip_distance_km"]].head(7)

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_distance_km
0,-73.973320,40.763805,-73.981430,40.743835,2.323989
1,-73.986862,40.719383,-73.998886,40.739201,2.426114
2,-73.982524,40.751260,-73.979654,40.746139,0.618822
3,-73.981160,40.767807,-73.990448,40.751635,1.961648
4,-73.966046,40.789775,-73.988565,40.744427,5.388992
5,-73.960983,40.765547,-73.979177,40.740053,3.223561
6,-73.949013,40.773204,-73.959622,40.770893,0.929893


In [39]:
train_df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance_km,trip_distance_2norm,trip_distance_1norm,log_trip_ditance
0,2009-06-15 17:26:21.0000001,4.50,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1,1.031088,0.000165,0.000205,0.030614
1,2010-01-05 16:52:16.0000002,16.90,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1,8.452786,0.001391,0.001876,2.134496
2,2011-08-18 00:35:00.00000049,5.70,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,1.389961,0.000239,0.000335,0.329276
3,2012-04-21 04:30:42.0000001,7.70,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1,2.800149,0.000442,0.000513,1.029673
4,2010-03-09 07:51:00.000000135,5.30,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,1.999784,0.000340,0.000475,0.693039
5,2011-01-06 09:50:45.0000002,12.10,2011-01-06 09:50:45,-74.000964,40.731630,-73.972892,40.758233,1,3.788428,0.000675,0.000954,1.331951
6,2012-11-20 20:35:00.0000001,7.50,2012-11-20 20:35:00,-73.980002,40.751662,-73.973802,40.764842,1,1.556295,0.000254,0.000338,0.442308
7,2012-01-04 17:22:00.00000081,16.50,2012-01-04 17:22:00,-73.951300,40.774138,-73.990095,40.751048,1,4.156749,0.000788,0.001080,1.424733
8,2012-12-03 13:10:00.000000125,9.00,2012-12-03 13:10:00,-74.006462,40.726713,-73.993078,40.731628,1,1.253625,0.000249,0.000319,0.226039
9,2009-09-02 01:11:00.00000083,8.90,2009-09-02 01:11:00,-73.980658,40.733873,-73.991540,40.758138,2,2.850521,0.000464,0.000613,1.047502


## 3. train & validate

In [40]:
X = train_df[['pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count','trip_distance_km', 'trip_distance_km','trip_distance_1norm', 'trip_distance_2norm']].values
Y = train_df[['fare_amount']].values

X_real_test = test_df[['pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count', 'trip_distance_km', 'trip_distance_km','trip_distance_1norm', 'trip_distance_2norm']].values

training_data = (X, Y)

In [43]:
# import pipeline
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import warnings; warnings.simplefilter('ignore')
import numpy as np
import pandas as pd


# Preprocess
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import datasets


# import regressors
# for regressors, score function is R2-score in sklearn.metrics
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct, WhiteKernel
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor

# import MSE
from sklearn.metrics import mean_squared_error


# Result
col = ['model', 'MSE', 'R2']
regResult = pd.DataFrame(columns=col)


# Regressor List
k_of_kNN = 1
regressornames = ["SGD", str(k_of_kNN)+" Nearest Neighbors",  str(k_of_kNN+2)+" Nearest Neighbors",#"Linear SVR", 
                  #"RBF SVR", 
         #"RBF Gaussian Process", "DotKernel Gaussian Process",
         "Decision Tree", "Random Forest", "AdaBoost", 
                  #"Neural Net",
         "Ridge1", "Ridge2"]#, "Ridge3"]#, "Ridge4"]#, "Ridge5"]

regressors = [
    SGDRegressor(max_iter=100, tol=1e-3),
    KNeighborsRegressor(k_of_kNN), 
    KNeighborsRegressor(k_of_kNN+2), 
#    SVR(kernel='linear', C=1e3),
#    SVR(kernel='rbf', C=1e3, gamma=0.1),
#    GaussianProcessRegressor(kernel=1.0 * RBF(1.0)),
#    GaussianProcessRegressor(kernel=DotProduct() + WhiteKernel()),
    DecisionTreeRegressor(max_depth=5),
    RandomForestRegressor(max_depth=10, n_estimators=10, max_features=2), 
    AdaBoostRegressor(),
#    MLPRegressor(hidden_layer_sizes=(5,),
#                   learning_rate='adaptive',
#                   max_iter=10000,
#                   learning_rate_init=0.01,
#                   alpha=0.01),
    make_pipeline(PolynomialFeatures(1), Ridge()),
    make_pipeline(PolynomialFeatures(2), Ridge())#,
#    make_pipeline(PolynomialFeatures(3), Ridge())#,
#    make_pipeline(PolynomialFeatures(4), Ridge())#,
#    make_pipeline(PolynomialFeatures(5), Ridge())
    ]

# Dataset List

datasets = [training_data]

# iterate over datasets
for ds_cnt, ds in enumerate(datasets):
    X, y = ds
    regResult = pd.DataFrame(columns=col)
    
    # Standardize
    #X = StandardScaler().fit_transform(X)
    
    # Data Split
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.3, random_state=42)
    
    #X_real_test = test_df[['pickup_longitude',
    #   'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
    #   'passenger_count']].values
    
    # iterate over regressors
    for name, regressor in zip(regressornames, regressors):
        regressor.fit(X_train, y_train)
        score = regressor.score(X_test, y_test)
        y_pred = regressor.predict(X_test)
        #y_pred_real_test = regressor.predict(X_real_test)
        print(name, "score: ", score)
        regResult = regResult.append({'model': name, 
                          'MSE': mean_squared_error(y_test, y_pred), 
                          'R2': score}, 
                          ignore_index=True)        
    display(regResult)
    print('-'*40)

SGD score:  -1.3085855063341823e+22
1 Nearest Neighbors score:  0.637038378619565
3 Nearest Neighbors score:  0.7318732813085516
Decision Tree score:  0.7624976961196179
Random Forest score:  0.7996600560434886
AdaBoost score:  -3.5673451223489363
Ridge1 score:  0.5410708778064757
Ridge2 score:  0.6759665857440245


,model,MSE,R2
0,SGD,1.204996e+24,-1.308586e+22
1,1 Nearest Neighbors,3.342290e+01,6.370384e-01
2,3 Nearest Neighbors,2.469014e+01,7.318733e-01
3,Decision Tree,2.187013e+01,7.624977e-01
4,Random Forest,1.844807e+01,7.996601e-01
5,AdaBoost,4.205787e+02,-3.567345e+00
6,Ridge1,4.225996e+01,5.410709e-01
7,Ridge2,2.983824e+01,6.759666e-01


----------------------------------------


## Predict

In [22]:
import lightgbm as lgb

In [23]:
#regressor = KNeighborsRegressor(k_of_kNN+2)
#regressor =  RandomForestRegressor(max_depth=10, n_estimators=10, max_features=3)
#regressor =  make_pipeline(PolynomialFeatures(2), Ridge())

import lightgbm as lgb
regressor = lgb.LGBMRegressor()

regressor.fit(X_train, y_train)
y_pred_test = regressor.predict(X_real_test)

In [24]:
pd.Series(y_pred_test.flatten()).unique()
##y_pred_test

array([ 8.53588969,  8.73944259,  5.11022557, ..., 53.02670288,
       20.69259595,  6.69333038])

## Write to csv

In [25]:
subm_df = pd.read_csv('sample_submission.csv')
subm_df['fare_amount'] = y_pred_test.flatten()
subm_df.to_csv('submission.csv', index=False)

#holdout = pd.DataFrame({'key': test_df.index, 'fare_amount': y_pred_test.flatten()}, columns = ['key', 'fare_amount'])
#holdout


In [357]:
#holdout.to_csv('submission.csv', index=False)